In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
device  = torch.device(
    "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Using the device: {device}")
print(f"mps is Apple integrated GPU ")

Using the device: mps
mps is Apple integrated GPU 


In [3]:
from torchvision.datasets import CIFAR100
import torchvision.transforms as transforms


transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean =[0.485, 0.456, 0.406],
        std = [0.229, 0.224, 0.225]
    )
                                ])

In [4]:
train_ds = CIFAR100(root="./data", train=True, download=True, transform=transform)
test_ds  = CIFAR100(root="./data", train=False, download=True, transform=transform)

/opt/anaconda3/envs/ml/lib/python3.11/site-packages/torchvision/datasets/cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")


In [5]:
train_dataloader = DataLoader(
    train_ds,
    batch_size =8,
    shuffle=True,
    num_workers=4,
    persistent_workers=True
)
test_dataloader = DataLoader(
    test_ds,
    batch_size =8,
    shuffle=False
)

In [6]:
class VGG16(nn.Module):
    def __init__(self, num_classes=100):
        super().__init__()
        self.feature_extraction = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64,kernel_size=3,stride=1,padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding = 1, bias=False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),
            
            
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(in_channels = 128, out_channels=256, kernel_size = 3, padding = 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding = 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding = 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(in_channels = 256, out_channels=512, kernel_size = 3, padding = 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding = 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding = 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(in_channels = 512, out_channels=512, kernel_size = 3, padding = 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding = 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding = 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2)
                 
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        
        
        self.classifier = nn.Sequential(
            nn.Linear(in_features=512, out_features=4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
        
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
        
        
            nn.Linear(in_features=4096, out_features = num_classes)
    )
    def forward(self, x):
        x = self.feature_extraction(x)
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = self.classifier(x)
        return x

In [7]:
model = VGG16().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.01,
    momentum=0.9,
    weight_decay=5e-4
)

EPOCHS = 5


In [8]:
from tqdm import tqdm
train_losses = []
train_accs = []

for EPOCH in range(EPOCHS):

    model.train()

    running_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(train_dataloader, desc=f"Epoch {EPOCH+1}/{EPOCHS}"):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)

        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_dataloader)
    epoch_acc = correct / total

    train_losses.append(epoch_loss)
    train_accs.append(epoch_acc)

    print(f"Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")

Epoch 1/5: 100%|██████████| 6250/6250 [44:02<00:00,  2.37it/s]


Loss: 4.6093 | Acc: 0.0091


Epoch 2/5: 100%|██████████| 6250/6250 [47:00<00:00,  2.22it/s]


Loss: 4.6087 | Acc: 0.0098


Epoch 3/5: 100%|██████████| 6250/6250 [46:38<00:00,  2.23it/s]


Loss: 4.6087 | Acc: 0.0090


Epoch 4/5: 100%|██████████| 6250/6250 [47:35<00:00,  2.19it/s]


Loss: 4.6090 | Acc: 0.0096


Epoch 5/5: 100%|██████████| 6250/6250 [47:22<00:00,  2.20it/s]

Loss: 4.6089 | Acc: 0.0092


In [10]:
from tqdm import tqdm

model.eval()   # switch to evaluation mode

test_loss = 0
correct = 0
total = 0

with torch.no_grad():   # disable gradients (important)

    for images, labels in tqdm(test_dataloader, desc="Testing"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        loss = loss_fn(outputs, labels)
        test_loss += loss.item()

        _, preds = torch.max(outputs, 1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)


test_loss /= len(test_dataloader)
test_acc = correct / total

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Acc : {test_acc:.4f}")

Testing: 100%|██████████| 1250/1250 [02:20<00:00,  8.88it/s]

Test Loss: 4.6071
Test Acc : 0.0100
